In [13]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

In [14]:
df = pd.read_csv("fatwa-mui.csv")
df.head()

,No,Judul Fatwa,Tema Fatwa,Nomor Fatwa,Tgl Ditetapkan
0,1,PANDUAN PENYELENGGARAAN IBADAH DI BULAN RAMADA...,Sosial Kemasyarakatan,KEPUTUSAN IJTIMA ULAMA KOMISI FATWA SE-INDONES...,16 December 2003
1,2,PENGGUNAAN MIKROBA DAN PRODUK MIKROBIAL DALAM ...,POM Iptek,01 Tahun 2010,19 January 2010
2,3,AIR DAUR ULANG,POM Iptek,02 Tahun 2010,27 January 2010
3,4,KIBLAT,Ibadah,03 Tahun 2010,1 February 2010
4,5,ARAH KIBLAT,Ibadah,05 Tahun 2010,1 July 2010


In [15]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import spacy
from nltk.corpus import stopwords

nlp = spacy.blank('id')
factory = StemmerFactory()
stemmer = factory.create_stemmer()
indonesia_s = stopwords.words('indonesian')

def preprocess(text):
    no_punctuation = re.sub(r'[^\w\s]','', text)
    no_number = re.sub(r'\d+', "", no_punctuation)
    no_single_letter = ' '.join( [w for w in no_number.split() if len(w)>1] )
    stem_output = stemmer.stem(no_single_letter)
    doc = nlp(stem_output)
    no_stop_words = [token.text for token in doc if not token.is_stop]
    return " ".join(no_stop_words) 

In [16]:
def calc_euclidian_distance(array1, array2):
    sum_sq = np.sum(np.square(array1 - array2))
    return np.sqrt(sum_sq)

In [17]:
n = len(df)
empty_array = np.zeros((n, n))

In [18]:
corpus = []
for i in range(n):
    corpus.append(preprocess(df['Judul Fatwa'][i]));
print(corpus)

['pandu selenggara ibadah ramadan syawal', 'mikroba produk mikrobial produk pangan', 'air daur ulang', 'kiblat', 'arah kiblat', 'vaksin meningitis jemaah haji umrah', 'kopi luwak', 'amil zakat', 'pensucian alat produksi kena najis mutawassithah najis air', 'pensucian ekstrak ragi yeast ekstract sisa olah bir brewer yeast', 'hukum zakat harta haram', 'salur harta zakat bentuk aset kelola', 'tari pelihara salur harta zakat', 'hukum warna makan minum serangga cochineal', 'badal thawaf ifadhah laksana thawaf ifadhah orang', 'sembelih hewan dam haji tamattu tanah haram', 'sarang burung walet', 'duduk anak hasil zina laku hadap', 'semir rambut', 'hukum konsumsi bekicot', 'penyalahgunaan formalin bahan bahaya tangan olah ikan', 'manfaat bekicot non pangan', 'bulu rambut tanduk hewan halal sembelih syar bahan pangan obat obat kosmetika', 'plasenta hewan halal bahan obat', 'hukum hewan ternak pakan barang najis', 'istri', 'standar halal produk kosmetika', 'shellac bahan pangan obatobatan kosmet

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
v.fit(corpus)
transform_output = v.transform(corpus)

In [20]:
print(v.vocabulary_)

{'pandu': 523, 'selenggara': 658, 'ibadah': 259, 'ramadan': 610, 'syawal': 709, 'mikroba': 454, 'produk': 585, 'mikrobial': 455, 'pangan': 524, 'air': 8, 'daur': 158, 'ulang': 779, 'kiblat': 347, 'arah': 41, 'vaksin': 794, 'meningitis': 449, 'jemaah': 310, 'haji': 232, 'umrah': 783, 'kopi': 366, 'luwak': 418, 'amil': 32, 'zakat': 822, 'pensucian': 537, 'alat': 16, 'produksi': 586, 'kena': 335, 'najis': 480, 'mutawassithah': 477, 'ekstrak': 189, 'ragi': 605, 'yeast': 819, 'ekstract': 188, 'sisa': 683, 'olah': 504, 'bir': 104, 'brewer': 109, 'hukum': 255, 'harta': 241, 'haram': 239, 'salur': 640, 'bentuk': 90, 'aset': 46, 'kelola': 332, 'tari': 730, 'pelihara': 529, 'warna': 812, 'makan': 422, 'minum': 459, 'serangga': 664, 'cochineal': 137, 'badal': 60, 'thawaf': 748, 'ifadhah': 262, 'laksana': 390, 'orang': 510, 'sembelih': 659, 'hewan': 245, 'dam': 148, 'tamattu': 719, 'tanah': 721, 'sarang': 643, 'burung': 117, 'walet': 808, 'duduk': 175, 'anak': 34, 'hasil': 242, 'zina': 826, 'laku'

In [21]:
feature = transform_output.toarray()

In [22]:
for i in range(len(empty_array[i])):
    for j in range(len(empty_array[i])):
        if (i != j):
          if (empty_array[j][i] == 0):
            similarity = 1 / (1 + calc_euclidian_distance(feature[i],feature[j]))
            empty_array[i][j] = similarity
          else:
            continue
        else:
          empty_array[i][j] = 1

print(empty_array)

[[1.         0.41421356 0.41421356 ... 0.41421356 0.41421356 0.41421356]
 [0.         1.         0.41421356 ... 0.41421356 0.41421356 0.41421356]
 [0.         0.         1.         ... 0.41421356 0.41421356 0.41421356]
 ...
 [0.         0.         0.         ... 1.         0.41421356 0.4221139 ]
 [0.         0.         0.         ... 0.         1.         0.41421356]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [23]:
for i in range(len(empty_array[i])):
    for j in range(len(empty_array[i])):
        if(empty_array[i][j] > 0.7 and empty_array[i][j] < 1):
            empty_array[i][j] = 1
            print(df['Judul Fatwa'][i], ' | ', df['Judul Fatwa'][j], ' | ', empty_array[i][j])
        else:
            empty_array[i][j] = 0

PENGGUNAAN PLASENTA HEWAN HALAL UNTUK BAHAN OBAT  |  PENGGUNAAN PLASENTA HEWAN HALAL UNTUK BAHAN KOSMETIKA OBAT LUAR  |  1.0
PENYERANGAN AMERIKA SERIKAT DAN SEKUTUNYA TERHADAP IRAK  |  FATWA PENYERANGAN AMERIKA SERIKAT DAN SEKUTUNYA TERHADAP IRAK  |  1.0


In [24]:
print(df['Judul Fatwa'][7], ' | ', df['Judul Fatwa'][345], empty_array[7][345])
print(df['Judul Fatwa'][14], ' | ', df['Judul Fatwa'][240], empty_array[14][241])
print(df['Judul Fatwa'][23], ' | ', df['Judul Fatwa'][56], empty_array[23][56])
print(df['Judul Fatwa'][60], ' | ', df['Judul Fatwa'][306], empty_array[60][306])
print(df['Judul Fatwa'][80], ' | ', df['Judul Fatwa'][314], ' | ', empty_array[80][314])
print(df['Judul Fatwa'][84], ' | ', df['Judul Fatwa'][191], ' | ', empty_array[84][191])
print(df['Judul Fatwa'][92], ' | ', df['Judul Fatwa'][194], ' | ', empty_array[92][194])
print(df['Judul Fatwa'][96], ' | ', df['Judul Fatwa'][234], ' | ', empty_array[96][234])
print(df['Judul Fatwa'][154], ' | ', df['Judul Fatwa'][155], ' | ', empty_array[154][155])
print(df['Judul Fatwa'][203], ' | ', df['Judul Fatwa'][205], ' | ', empty_array[203][205])

AMIL ZAKAT  |  PENGELOLAAN ZAKAT 0.0
BADAL THAWAF IFADHAH (PELAKSANAAN THAWAF IFADHAH OLEH ORANG LAIN)  |  TASWIYAH AL-MANHAJ (PENYAMAAN POLA PIKIR DALAM MASALAH-MASALAH KEAGAMAAN) 0.0
PENGGUNAAN PLASENTA HEWAN HALAL UNTUK BAHAN OBAT  |  PENGGUNAAN ALKOHOL / ETANOL UNTUK BAHAN OBAT 0.0
TRANSPLANTASI ORGAN DAN ATAU / JARINGAN TUBUH UNTUK DIRI SENDIRI  |  TRANSPLANTASI ORGAN DAN/ATAU JARINGAN TUBUH DARI PENDONOR HIDUP UNTUK ORANG LAIN 0.0
PENDAFTARAN HAJI SAAT USIA DINI  |  PENDAFTARAN HAJI USIA DINI  |  0.0
PRODUK VAKSIN COVID-19 DARI SINOVAC LIFE SCIENCES CO. LTD, CHINA DAN PT. BIO FARMA (PERSERO)  |  PRODUK VAKSIN COVID-19 DARI PT. BIOTIS PHARMACEUTICALS INDONESIA  |  0.0
HUKUM PENGGUNAAN VAKSIN COVID-19 PRODUK SINOPHARM CNBG CHINA  |  HUKUM VAKSIN COVID-19 PRODUKSI CANSINO BIOLOGICS INC. CHINA  |  0.0
HUKUM DAN PANDUAN PELAKSANAAN IBADAH KURBAN SAAT KONDISI WABAH PENYAKIT MULUT DAN KUKU  |  HUKUM DAN PANDUAN PELAKSANAAN IBADAH KURBAN SAAT MEREBAKNYA PENYAKIT LSD DAN PPR PADA HEWAN KU

In [25]:
labeling = np.genfromtxt("labeling_fatwa2.csv", delimiter=",")
predict = empty_array.flatten()
label = labeling.flatten()

In [26]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

precision = precision_score(label, predict)
accuracy = accuracy_score(label, predict)
recall = recall_score(label, predict)
f1score = f1_score(label, predict)

print('Precision ', precision)
print('Accuracy', accuracy)
print('Recal', recall)
print('F1', f1score)

Precision  1.0
Accuracy 0.9991551246537396
Recal 0.016129032258064516
F1 0.031746031746031744
